In [ ]:
!pip install impyute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import datetime
import math
from collections import defaultdict
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

import seaborn as sns
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from impyute.imputation.cs import mice
from sklearn.linear_model import BayesianRidge
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, 
RepeatedStratifiedKFold, StratifiedKFold)
from sklearn.model_selection import StratifiedGroupKFold, GroupShuffleSplit, permutation_test_score
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, scale
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score,roc_curve, f1_score, precision_score, recall_score, silhouette_score
from sklearn.inspection import permutation_importance

import warnings
from sklearn.exceptions import ConvergenceWarning

# import datawig
# Machine Learning Algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import BayesianRidge

import lightgbm as lgb
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#pd.options.display.max_rows = 999
pd.set_option("display.max_columns", None)

/home/avalanche/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Imputation Functions

In [ ]:
def mice_imputation(df_cleaned):
    
    #Preparing measurement values by putting them into new df to impute them later
    X = df_cleaned[df_cleaned.columns[8:]]

    imputed = mice(X.values,)
    df_mice = pd.DataFrame(imputed)
    
    # Merging non-measurement features with non-outlier dataframe
    #df_mice.columns = df_cleaned.columns[9:]
    df_mice = pd.concat([df_cleaned[df_cleaned.columns[:8]], df_mice], ignore_index=True, axis=1)
    df_mice.columns = df_cleaned.columns
    return df_mice

In [ ]:
def iter_imputation(df_cleaned,estimator=BayesianRidge(), max_iter=10,iter_verbose=0,verbose=False):
    df_cleaned.reset_index(drop=True,inplace=True)
    data_iter =  df_cleaned.iloc[:,9:]
    # print total missing
    if verbose:
        print('Missing: %d' % df_cleaned.isnull().sum().sum())
    # define imputer
    imputer = IterativeImputer(n_nearest_features=3, 
                               imputation_order='ascending', 
                               verbose=iter_verbose,
                               max_iter=max_iter,
                              estimator=estimator)

    # fit and transform the dataset
    Xtrans = imputer.fit_transform(data_iter)
    # print total missing
    if verbose:
        print('Missing: %d' % np.count_nonzero(np.isnan(Xtrans)))
    
    # Merging non-measurement features with filled measurement into df

    df_Xtrans= pd.DataFrame(Xtrans)             
    df_iter_imputation = pd.concat([df_cleaned[df_cleaned.columns[:9]], df_Xtrans], ignore_index=True, axis=1)
    df_iter_imputation.columns = df_cleaned.columns
    
    
    return df_iter_imputation

In [ ]:
# def iter_imputation(df_cleaned):
    
#     data_iter =  df_cleaned.drop(df_cleaned.iloc[:, 0:9], axis= 1)
#     # split into input and output elements
#     df_cleaned_values = data_iter.values
#     ix = [i for i in range(data_iter.shape[1]) if i != 0 ]
#     # X, y = df_cleaned_values[:, ix], df_cleaned_values[:, 0]
#     X, y = df_cleaned_values, df_cleaned_values[:, 0]

#     # print total missing
#     print('Missing: %d' % df_cleaned.isnull().sum().sum())
#     # define i,mputer
#     imputer = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=3, imputation_order='ascending',random_state=0)
#     # fit on the dataset
#     imputer.fit(X)
#     # transform the dataset
#     Xtrans = imputer.transform(X)
#     # print total missing
#     print('Missing: %d' % np.count_nonzero(np.isnan(Xtrans)))
    
#     # Merging non-measurement features with filled measurement into df

#     df_Xtrans= pd.DataFrame(Xtrans)             
#     df_iter_imputation = pd.concat([df_cleaned[df_cleaned.columns[:9]], df_Xtrans], ignore_index=True, axis=1)
#     df_iter_imputation.columns = df_cleaned.columns
    
    
#     return df_iter_imputation

In [ ]:
def iter_imputation_v2(df_out_notLike,estimator=BayesianRidge(),max_iter=10,iter_verbose=0,verbose=False):
# Some participants' hemodynamic measurements, 
# e.g. Goxy15, is totally empty so we need to first impute first row of it with its neighbours
# to not get eror in iterative imputation
    if not verbose:
        warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
    else:
        warnings.filterwarnings(action='default', category=ConvergenceWarning)


    part_no = df_out_notLike['Part_No'].drop_duplicates().to_list()
    df_list = []
    count = 1
    for part in part_no:
        if verbose:
            print(f'Iteration: {count} Part_no:{part}')
        df_temp = df_out_notLike[df_out_notLike['Part_No'] == part]
        # assign a value to the first value of a column where it has all values as NaN
        columns = df_temp.columns
        for colName in columns:
            if df_temp[colName].isna().sum() >= df_temp.shape[0]:

                if colName[-2:] == '15' or colName[-2:] == '16':
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)-2:columns.get_loc(colName)].mean()
                    if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                        df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)-2:
                                                                                columns.get_loc(colName)].mean().mean()
                elif (int(colName[-1]) > 0 and int(colName[-1]) < 10) or (int(colName[-2:]) > 9 and int(colName[-2:]) < 14):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)+1:
                                                                        columns.get_loc(colName)+3].mean()
                    if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                        df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)+1:
                                                                        columns.get_loc(colName)+5].mean().mean()
                if verbose:
                    print(f'ColName={colName} and ParticipantNo={part}')



        df_temp = iter_imputation(df_temp.reset_index(drop=True),
                                  estimator=estimator,
                                  max_iter=max_iter,
                                  iter_verbose=iter_verbose,
                                  verbose=verbose)
        df_list.append(df_temp)
        count += 1
    df_temp_2 = df_list.pop(0)
    for part in df_list:
        df_temp_2 = pd.concat([df_temp_2,part])
    df_temp_2 = df_temp_2.reset_index(drop=True)
    return df_temp_2

In [ ]:
def iter_imputation_v3(df_out_notLike, estimator=BayesianRidge(), max_iter=10, iter_verbose=0, verbose=False):
# Some participants' hemodynamic measurements,
# e.g. Goxy15, is totally empty so we need to first impute first row of it with its neighbours
# to not get eror in iterative imputation
    if not verbose:
        warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
        pd.options.mode.chained_assignment = None
    else:
        warnings.filterwarnings(action='default', category=ConvergenceWarning)
        pd.options.mode.chained_assignment = 'warn'

    part_no = df_out_notLike['Part_No'].drop_duplicates().to_list()
    df_list = []
    count = 1
    for part in part_no:
        if verbose:
            print(f'Iteration: {count} Part_no:{part}')
        df_temp = df_out_notLike.loc[df_out_notLike['Part_No'] == part]
        # assign a value to the first value of a column where it has all values as NaN
        columns = df_temp.columns
        for colName in columns:
            if df_temp[colName].isna().sum() >= df_temp.shape[0]:

                if colName[-2:] == '16' or colName[-1:] == '8':
                    loc = columns.get_loc(colName)
                    df_temp.loc[:, colName] = df_temp.loc[:,[columns[loc-1], columns[loc-2]]].mean(axis=1)
 
                elif colName[-2:] == '15' or colName[-1:] == '7':
                    loc = columns.get_loc(colName)
                    df_temp.loc[:, colName] = df_temp.loc[:, [columns[loc+1], columns[loc-2]]].mean(axis=1)

                elif any(colName[-2:] in s for s in ('11','12','13','14')):
                    loc= columns.get_loc(colName)
                    df_temp.loc[:, colName] = df_temp.loc[:, [columns[loc+2], columns[loc-2]]].mean(axis=1)

                elif colName[-2:] == '10' or int(colName[-1:]) == 2:
                    loc= columns.get_loc(colName)
                    df_temp.loc[:, colName] = df_temp.loc[:, [columns[loc+2], columns[loc-1]]].mean(axis=1)

                elif int(colName[-1:]) == 9 or int(colName[-1:]) == 1:
                    loc= columns.get_loc(colName)
                    df_temp.loc[:, colName] = df_temp.loc[:, [columns[loc+2], columns[loc+1]]].mean(axis=1)

                elif int(colName[-1:]) >  2 and int(colName[-1:]) < 7:
                    loc = columns.get_loc(colName)
                    df_temp.loc[:, colName]= df_temp.loc[:, [columns[loc+2], columns[loc-2]]].mean(axis=1)

       
                if verbose:
                    print(f'ColName={colName} and ParticipantNo={part}')



        df_temp= iter_imputation(df_temp.reset_index(drop=True),
                                  estimator=estimator,
                                  max_iter=max_iter,
                                  iter_verbose=iter_verbose,
                                  verbose=verbose)
        df_list.append(df_temp)
        count += 1
    df_temp_2= df_list.pop(0)
    for part in df_list:
        df_temp_2 = pd.concat([df_temp_2, part])
    df_temp_2= df_temp_2.reset_index(drop=True)
    pd.options.mode.chained_assignment = 'warn'
    return df_temp_2

# Outlier Count Per Feature

In [ ]:
def outlierCountbyFeature(df):
    
    Q1 = df[df.columns[9:]].quantile(0.25)
    Q3 = df[df.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1
    
    df_last_mea = df[df.columns[9:]]
    df_last_mea = df_last_mea.fillna(df_last_mea.mean())
    outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) | (df_last_mea > (Q3 + 1.5 * IQR)))
    
    outlier_count = outlier_matrix.sum(axis=0)
    outlier_percent = 100 * outlier_matrix.sum(axis=0) / df_last_mea.shape[0]
    
    # Df
    outlier_df = pd.concat([outlier_count, outlier_percent],
                              axis=1)
    
    outlier_df_final = outlier_df.rename(
        columns={0: 'Outlier Count', 1: '% of Total Values'})
    
    # Sort the table by percentage of outlier count descending
    outlier_df_final = (outlier_df_final[
        outlier_df_final.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))
    return outlier_df_final

# Outlier Count Per Fow

In [ ]:
def outlierCountbyRow(df):
    
    Q1 = df[df.columns[9:]].quantile(0.25)
    Q3 = df[df.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1
    
    df_last_mea = df[df.columns[9:]]
    df_last_mea = df_last_mea.fillna(df_last_mea.mean())
    outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) | (df_last_mea > (Q3 + 1.5 * IQR)))
    outlier_count = outlier_matrix.sum(axis=1)
    outlier_percent = 100 * outlier_matrix.sum(axis=1) / df_last_mea.shape[1]
    
    # Df

    outlier_df = pd.concat([outlier_count, outlier_percent],
                              axis=1)
    
    outlier_df_final = outlier_df.rename(
        columns={0: 'Outlier Feature Count', 1: '% of Total Values'})
    
    # Sort the table by percentage of outlier count descending
    outlier_df_final = (outlier_df_final[
        outlier_df_final.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))
    return outlier_df_final

In [ ]:
def outlierCountbyRow_and_Activity(df):
    
    Q1 = df[df.columns[9:]].quantile(0.25)
    Q3 = df[df.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1
    
    df_last_mea = df[df.columns[9:]]
    df_last_mea = df_last_mea.fillna(df_last_mea.mean())

    outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) | (df_last_mea > (Q3 + 1.5 * IQR)))
    outlier_count = outlier_matrix.sum(axis=1)
    outlier_percent = 100 * outlier_matrix.sum(axis=1) / df_last_mea.shape[1]

    outlier_matrix_higherActivity = ((df_last_mea > (Q3 + 1.5 * IQR)))
    outlier_count_higherActivity = outlier_matrix_higherActivity.sum(axis=1)
    outlier_percent_higherActivity = 100 * outlier_matrix_higherActivity.sum(axis=1) / df_last_mea.shape[1]

    outlier_matrix_lowerActivity = ((df_last_mea < (Q1 - 1.5 * IQR)))
    outlier_count_lowerActivity = outlier_matrix_lowerActivity.sum(axis=1)
    outlier_percent_lowerActivity = 100 * outlier_matrix_lowerActivity.sum(axis=1) / df_last_mea.shape[1]
    
    # Df
    outlier_df_new = pd.concat([outlier_count_higherActivity,outlier_percent_higherActivity, outlier_count_lowerActivity, outlier_percent_lowerActivity, outlier_count, outlier_percent],
                              axis=1)  
    outlier_df_final = outlier_df_new.rename(
        columns={0: 'Outlier Feature Count(Higher Activity)', 1: 'Higher% of Total Values', 2:'Outlier Feature Count(Lower Activity)', 3: 'Lower% of Total Values',4: 'Outlier Feature Count(All)',5: '% of Total Values'})
    
    # Sort the table by percentage of outlier count descending
    outlier_df_final = (outlier_df_final[
        outlier_df_final.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))
    return outlier_df_final

# Missing Values Per Row Indices

In [ ]:
def missingValuesRows(df):
    # Total missing valuesfvgcvg
    mis_val = df.isnull().sum(axis=1)

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum(axis=1) / df.shape[1]

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent],
                              axis=1)

    # Rename the columns
    mis_val_table_ren_rows = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_rows = (mis_val_table_ren_rows[
        mis_val_table_ren_rows.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[0]) + " columns.\n"
          "There are " + str(mis_val_table_ren_rows.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_rows

# Missing Value Per Features

In [ ]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent],
                              axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = (mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 0] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns



# Outlier Removal

In [ ]:
def OutlierRemoval_OutlierCap_byParticipant(df_last, outliersbyRow=18, perc=15):
    c = df_last.columns[9:]
    participant = df_last['Part_No'].unique()
    df_out_final=pd.DataFrame(columns=df_last.columns)
    for i in participant:

        Q1 = df_last.loc[df_last.Part_No == i][c].quantile(0.25)
        Q3 = df_last.loc[df_last.Part_No == i][c].quantile(0.75)
        IQR = Q3 - Q1

        df_last_mea = df_last.loc[df_last.Part_No == i][c]
#         df_last_mea = df_last_mea.fillna(df_last_mea.mean())

        outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) |
                          (df_last_mea > (Q3 + 1.5 * IQR)))
    
        # capping outliers
        df_cap = cap_outliers(df_last.loc[df_last.Part_No == i])

        perc_temp = 1
        outliersbyRow_temp = 100
        while perc_temp < perc:
            out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow_temp).sum()
            perc_temp = (out_count/df_last_mea.shape[0]*100).round(1)
            if perc_temp > perc:
                outliersbyRow_temp+=1
                out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow_temp).sum()
            else:
                outliersbyRow_temp-=1
        outliersbyRow=outliersbyRow_temp
            

        print(
            f'{i}: Row count to be removed:{out_count}, Total Rows: {df_last_mea.shape[0]} Removing Percentage:{(out_count/df_last_mea.shape[0]*100).round(1)}%')

        # Removing rows which have > outliersbyRow outlier count
        df_cap = df_cap[outlier_matrix.sum(axis=1) < outliersbyRow]
        
        # add participant to final df
        df_out_final = pd.concat([df_out_final,df_cap])
             
    df_out_final.reset_index(inplace=True,drop=True)
    return df_out_final

In [ ]:
def OutlierRemoval_OutlierCap(df_last, outliersbyRow=18, perc=15):
    Q1 = df_last[df_last.columns[9:]].quantile(0.25)
    Q3 = df_last[df_last.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1

    df_last_mea = df_last[df_last.columns[9:]]
#     df_last_mea = df_last_mea.fillna(df_last_mea.mean())
    outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) |
                      (df_last_mea > (Q3 + 1.5 * IQR)))

    # capping outliers
    df_cap = cap_outliers(df_last)

    perc_temp = 1
    outliersbyRow_temp = 100
    while perc_temp < perc:
        out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow_temp).sum()
        perc_temp = (out_count/df_last_mea.shape[0]*100).round(1)
        if perc_temp > perc:
            outliersbyRow_temp += 1
            out_count = (outlier_matrix.sum(axis=1)
                         >= outliersbyRow_temp).sum()
        else:
            outliersbyRow_temp -= 1
    outliersbyRow = outliersbyRow_temp

#     out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow).sum()
    print(f'Row count to be removed:{out_count}, Total Rows: {df_last_mea.shape[0]}'
          f'Removing Percentage:{(out_count/df_last_mea.shape[0]*100).round(1)}%')

    # Removing outlier records
    df_cap = df_cap[outlier_matrix.sum(axis=1) < outliersbyRow]

    return df_cap

In [ ]:
def OutlierRemoval_OutlierTransformNan_byParticipant(df_last, outliersbyRow=18, perc=15):
    first_c = df_last.columns[:9]
    c = df_last.columns[9:]
    participant = df_last['Part_No'].unique()
    df_out_final=pd.DataFrame(columns=df_last.columns)
    for i in participant:

        Q1 = df_last.loc[df_last.Part_No == i][c].quantile(0.25)
        Q3 = df_last.loc[df_last.Part_No == i][c].quantile(0.75)
        IQR = Q3 - Q1

        df_last_mea = df_last.loc[df_last.Part_No == i][c]
#         df_last_mea = df_last_mea.fillna(df_last_mea.mean())

        outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) |
                          (df_last_mea > (Q3 + 1.5 * IQR)))

        perc_temp = 1
        outliersbyRow_temp = 100
        while perc_temp < perc:
            out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow_temp).sum()
            perc_temp = (out_count/df_last_mea.shape[0]*100).round(1)
            if perc_temp > perc:
                outliersbyRow_temp+=1
                out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow_temp).sum()
            else:
                outliersbyRow_temp-=1
        outliersbyRow=outliersbyRow_temp
            

#         out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow).sum()

        print(
            f'{i}: Row count to be removed:{out_count}, Total Rows: {df_last_mea.shape[0]} Removing Percentage:{(out_count/df_last_mea.shape[0]*100).round(1)}%')

        # Removing rows which have > outliersbyRow outlier count
        df_out = df_last_mea[outlier_matrix.sum(axis=1) < outliersbyRow]

        # Transforming the matrix as same as df_out to convert remaining outliers into Nan
        outlier_matrix = outlier_matrix[outlier_matrix.sum(
            axis=1) < outliersbyRow]
        df_out = df_out.mask((outlier_matrix))
        df_out = pd.concat([df_last.loc[df_last.Part_No == i][first_c],
                           df_out], join='inner', axis=1)
        # add participant to final df
        df_out_final = pd.concat([df_out_final,df_out])
    df_out_final.reset_index(inplace=True,drop=True)
    return df_out_final

In [ ]:
def OutlierRemoval_OutlierTransformNan(df_last, outliersbyRow=18):
    Q1 = df_last[df_last.columns[9:]].quantile(0.25)
    Q3 = df_last[df_last.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1

    df_last_mea = df_last[df_last.columns[9:]]
    df_last_mea = df_last_mea.fillna(df_last_mea.mean())
    outlier_matrix = ((df_last_mea < (Q1 - 1.5 * IQR)) |
                      (df_last_mea > (Q3 + 1.5 * IQR)))

    out_count = (outlier_matrix.sum(axis=1) >= outliersbyRow).sum()
    print(f'Row count to be removed:{out_count}, Total Rows: {df_last_mea.shape[0]}'
          f'Removing Percentage:{(out_count/df_last_mea.shape[0]*100).round(1)}%')
    df_last_mea = df_last[df_last.columns[9:]]

    # Removing rows which have > outliersbyRow outlier count
    df_out = df_last_mea[outlier_matrix.sum(axis=1) < outliersbyRow]

    # Transforming the matrix as same as df_out to convert outliers into Nan
    outlier_matrix = outlier_matrix[outlier_matrix.sum(axis=1) < outliersbyRow]
    NA_sum_init = df_out.isna().sum().sum()
    df_out = df_out.mask((outlier_matrix))
    NA_sum_fin = df_out.isna().sum().sum()
    print(f"Total masked values are: {NA_sum_fin-NA_sum_init}"
          f",{(NA_sum_fin-NA_sum_init)/NA_sum_init*100}.2f% of Initial NA, total NA:{NA_sum_fin}")
    df_out = pd.concat([df_last[df_last.columns[:9]],
                       df_out], join='inner', axis=1)
    return df_out

In [ ]:
def OutlierTransformNan(df_last):

    Q1 = df_last[df_last.columns[9:]].quantile(0.25)
    Q3 = df_last[df_last.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1

    # Outlier dataframe, True means it is an outlier
    df_out = (df_last[df_last.columns[9:]] < (Q1 - 1.5 * IQR)
              ) | (df_last[df_last.columns[9:]] > (Q3 + 1.5 * IQR))
    true_count = (df_out == True).sum(axis=1).sum()
    false_count = (df_out == False).sum(axis=1).sum()
    print("True count: {} False Count: {}".format(true_count, false_count))

    # to check if all values of a row has outlier
    print(df_out.shape, df_out[~df_out.all(1)].shape)

    # Transforming outliers into NaN
    df_last_mea = df_last[df_last.columns[9:]]
    df_out = df_last_mea.mask(
        ((df_last_mea < (Q1 - 1.5 * IQR)) | (df_last_mea > (Q3 + 1.5 * IQR))))

    # Adding non-measurement columns
    df_last_out = pd.concat(
        [df_last[df_last.columns[:9]], df_out], ignore_index=True, axis=1)
    df_last_out.columns = df_last.columns

    return df_last_out

In [ ]:
def cap_outliers(df, zscore_threshold=4, verbose=False):
    '''Caps outliers to closest existing value within threshold (Z-score).'''

    c = df.columns[9:]
    participant = df['Part_No'].unique()
    for i in participant:
        # mean_val = series.loc[series.Part_No == i][c].mean()
        # std_val = series.loc[series.Part_No == i][c].std()

        # lbound = mean_val - zscore_threshold * std_val
        # ubound = mean_val + zscore_threshold * std_val

        lbound = df.loc[df.Part_No == i][c].quantile(0.25)
        ubound = df.loc[df.Part_No == i][c].quantile(0.75)
        threshold = ubound - lbound

        df = df.copy()
        df.loc[df.Part_No == i, c] = df.loc[df.Part_No == i][c].clip(
            lbound - 1.5*threshold, ubound + 1.5*threshold, axis=1)

    return df

# Standardization

In [ ]:
def standardization_byParticipant(df):
    c = df.columns[:-1]
    participant = df['Part_No'].unique()
    df_final=pd.DataFrame(columns=c)
    for i in participant:
      scaler = StandardScaler()
      std = df.loc[df.Part_No == i][c]
      std = scaler.fit_transform(std)
      df_std = pd.DataFrame(std,columns=c)
      # print(df_std)
      # break
      df_final = pd.concat([df_final,df_std])
    df_final.reset_index(inplace=True,drop=True)
    return df_final

In [ ]:
def standardization_byRows(df):
  c = df.columns[:-1] 
  std = df[c].sub(df[c].mean(1), axis=0).div(df[c].std(1), axis=0)
  # df_std = pd.DataFrame(std,columns=c)
  return std

# FAA Transformation

In [ ]:
def avg_lr(df):
    df_avg_lr = df.copy()
    df_avg_lr.drop(df_avg_lr.iloc[:,9:], inplace=True, axis=1)

    df_avg_lr['Goxy_Avg'] = df.iloc[:,9:17].mean(axis=1) / df.iloc[:,17:25].mean(axis=1)
    df_avg_lr['Ghbr_Avg'] = df.iloc[:,25:33].mean(axis=1) / df.iloc[:,33:41].mean(axis=1)
    df_avg_lr['Ghbo_Avg'] = df.iloc[:,41:49].mean(axis=1) / df.iloc[:,49:57].mean(axis=1)
    df_avg_lr['Ghbt_Avg'] = df.iloc[:,57:65].mean(axis=1) / df.iloc[:,65:73].mean(axis=1)

    df_avg_lr['Koxy_Avg'] = df.iloc[:,73:81].mean(axis=1) / df.iloc[:,81:89].mean(axis=1)
    df_avg_lr['Khbr_Avg'] = df.iloc[:,89:97].mean(axis=1) / df.iloc[:,97:105].mean(axis=1)
    df_avg_lr['Khbo_Avg'] = df.iloc[:,105:113].mean(axis=1) / df.iloc[:,113:121].mean(axis=1)
    df_avg_lr['Khbt_Avg'] = df.iloc[:,121:129].mean(axis=1) / df.iloc[:,129:137].mean(axis=1)

    # df_avg_lr['Goxy_Avg9-16'] = df_cleaned.iloc[:,25:33].mean(axis=1)
    return df_avg_lr

# FAA Transformation with Creating a FAA Feature - 1 for left region and 0 for right region dominance

In [ ]:
def FAA_dominance(df):
    df_avg_lr = df.copy()
    
    df_avg_lr['Goxy_FAA_Index'] = df.iloc[:,9:17].mean(axis=1) - df.iloc[:,17:25].mean(axis=1)
    df_avg_lr['Koxy_FAA_Index'] = df.iloc[:,73:81].mean(axis=1) - df.iloc[:,81:89].mean(axis=1)

    # df_avg_lr['Goxy_Avg9-16'] = df_cleaned.iloc[:,25:33].mean(axis=1)
    return df_avg_lr

# Kmeans & GMM

In [ ]:
def amountByCluster(df,clusterNum):
    df_prc = pd.DataFrame(df_avgiter_GMM_onehot_cl2.iloc[:,-clusterNum:].sum())
    df_prc[1] = 0
    for i in range(0,clusterNum):
        df_prc.iloc[i,1] = df_prc.iloc[i,0] / df_prc.iloc[:,0].sum() * 100

    return df_prc.sort_values(by=0, ascending=False)


def gmm_js(gmm_p, gmm_q, n_samples=10**5):
    X = gmm_p.sample(n_samples)[0]
    log_p_X = gmm_p.score_samples(X)
    log_q_X = gmm_q.score_samples(X)
    log_mix_X = np.logaddexp(log_p_X, log_q_X)

    Y = gmm_q.sample(n_samples)[0]
    log_p_Y = gmm_p.score_samples(Y)
    log_q_Y = gmm_q.score_samples(Y)
    log_mix_Y = np.logaddexp(log_p_Y, log_q_Y)

    return np.sqrt((log_p_X.mean() - (log_mix_X.mean() - np.log(2))
            + log_q_Y.mean() - (log_mix_Y.mean() - np.log(2))) / 2)

def df_Kmeans_evaluation(df):
    
    df_age_sex = pd.read_csv(r'FNIRS_bilgi.csv')
#     df_age_sex = pd.read_csv(r'/content/drive/MyDrive/Master/Tez/FNIRS_bilgi.csv')
    df_age_sex.columns = ['Part_No', 'Gender', 'Age', 'Education']
    
    # Merging gender data with main data
    df_whole = pd.merge(df_age_sex, 
                          df, 
                          on ='Part_No', 
                          how ='inner')
    
    # Gender One Hot Encoding
    One_enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(One_enc.fit_transform(df_whole[['Gender']]).toarray())
    df_whole = df_whole.join(enc_df)
    df_whole.rename(columns = {0:'Male', 1:'Female'}, inplace = True)
    
    Z = df_whole.groupby(['Part_No'], as_index = False).mean()
    
#     s_scores = []
#     for i in range(2,10):
#         KMean = KMeans(n_clusters=i)
#         KMean.fit(Z.iloc[:,8:16])
#         label = KMean.predict(Z.iloc[:,8:16])
#         print(f'Silhouette Score(n={i}): {silhouette_score(Z.iloc[:,8:16], label)}')
#         s_scores.append(silhouette_score(Z.iloc[:,8:16], label))
#     plt.plot(np.arange(2,20), s_scores)
#     plt.title('Silhouette Score of K-Means')
#     plt.xlabel('Cluster Number')
#     plt.ylabel('Sİlhouette Score')
#     plt.show()

    # Silhouette Score
    n_components = np.arange(2, 20)
    models = [KMeans(n_clusters = n).fit(Z.iloc[:,9:]) for n in n_components]
    labels = [model.predict(Z.iloc[:,9:]) for model in models]
    plt.plot(n_components, [silhouette_score(Z.iloc[:,9:], label) for label in labels] )
    plt.title('Silhouette Score of K-Means')
    plt.xlabel('Cluster Number')
    plt.ylabel('Silhouette Score')
    plt.xticks(np.arange(1,20,1))
    plt.show()
    
def df_Kmeans_silhouette(df, clusters):
    
    df_age_sex = pd.read_csv(r'FNIRS_bilgi.csv')
#     df_age_sex = pd.read_csv(r'/content/drive/MyDrive/Master/Tez/FNIRS_bilgi.csv')
    df_age_sex.columns = ['Part_No', 'Gender', 'Age', 'Education']
    
    # Merging gender data with main data
    df_whole = pd.merge(df_age_sex, 
                          df, 
                          on ='Part_No', 
                          how ='inner')
    
    # Gender One Hot Encoding
    One_enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(One_enc.fit_transform(df_whole[['Gender']]).toarray())
    df_whole = df_whole.join(enc_df)
    df_whole.rename(columns = {0:'Male', 1:'Female'}, inplace = True)
    Z = df_whole.groupby(['Part_No'], as_index = False).mean()

    # K-means Clustering and its One Hot Encoding
    KMean = KMeans(n_clusters=clusters)
    KMean.fit(Z.iloc[:,9:])
    label = KMean.predict(Z.iloc[:,9:])
    Z['Clusters'] = label
    Z = Z[['Part_No']].join(Z['Clusters'])    
    df_whole = pd.merge(df_whole, 
                          Z, 
                          on ='Part_No', 
                          how ='inner')    
    One_enc_2 = OneHotEncoder(handle_unknown='ignore')
    enc_df_2 = pd.DataFrame(One_enc_2.fit_transform(df_whole[['Clusters']]).toarray())
    df_whole = df_whole.join(enc_df_2)    
#     df_whole.rename(columns = {0:'Cluster 1', 1:'Cluster 2'}, inplace = True)    
    df_whole.drop(['Clusters', 'Gender', 'Age', 'Education'], inplace = True, axis = 1)
    
    return df_whole    


def df_GMM_evaluation(df):
    
    df_age_sex = pd.read_csv(r'FNIRS_bilgi.csv')
#     df_age_sex = pd.read_csv(r'/content/drive/MyDrive/Master/Tez/FNIRS_bilgi.csv')
    df_age_sex.columns = ['Part_No', 'Gender', 'Age', 'Education']
    
    # Merging gender data with main data
    df_whole = pd.merge(df_age_sex, 
                          df, 
                          on ='Part_No', 
                          how ='inner')
    
    # Gender One Hot Encoding
    One_enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(One_enc.fit_transform(df_whole[['Gender']]).toarray())
    df_whole = df_whole.join(enc_df)
    df_whole.rename(columns = {0:'Male', 1:'Female'}, inplace = True)
    Z = df_whole.groupby(['Part_No'], as_index = False).mean()
    

    # Silhouette Score
#     n_components = np.arange(2, 20)
#     models = [GMM(n_components = n, covariance_type='full').fit(Z.iloc[:,9:]) for n in n_components]
#     labels = [model.predict(Z.iloc[:,9:]) for model in models]
#     plt.plot(n_components, [silhouette_score(Z.iloc[:,9:], label) for label in labels] )
#     plt.title('Silhouette Score of GMM')
#     plt.xlabel('Cluster Number')
#     plt.ylabel('Silhouette Score')
#     plt.xticks(np.arange(1,20,1))
#     plt.show()
    
    # BIC/AIC Score
    n_components_ = np.arange(2, 20)
    models = [GMM(n, covariance_type='full', random_state=0).fit(Z.iloc[:,9:])
          for n in n_components_]

    plt.plot(n_components_, [m.bic(Z.iloc[:,9:]) for m in models], label='BIC')
    plt.plot(n_components_, [m.aic(Z.iloc[:,9:]) for m in models], label='AIC')
    plt.legend(loc='best')
    plt.title('BIC AIC Evaluation')
    plt.xlabel('n_components')
    plt.ylabel('Score')
    plt.xticks(np.arange(1,20,1))
    plt.show()
    
    #Jensen Shannon Divergence Score/Distances
#     js_scores = []
#     for n in n_components_:
#         train, test = train_test_split(Z.iloc[:,9:], test_size=0.5)
#         norm = MinMaxScaler()
#         train_norm = norm.fit_transform(train).reshape(-1,1)
#         test_norm = norm.transform(test).reshape(-1,1)
#         gmm_train = GMM(n_components = n, covariance_type='full' ).fit(train_norm) 
#         gmm_test = GMM(n_components = n, covariance_type='full').fit(test_norm) 
#         js_scores.append(gmm_js(gmm_train, gmm_test))    
    
#     plt.plot(n_components, js_scores)
#     plt.title('Jensen Shannon Divergence GMM')
#     plt.xlabel('Cluster Number')
#     plt.ylabel('Distance')
#     plt.xticks(np.arange(1,20,1))
#     plt.show()
    

def df_GMM(df, clusters):
    
    df_age_sex = pd.read_csv(r'FNIRS_bilgi.csv')
#     df_age_sex = pd.read_csv(r'/content/drive/MyDrive/Master/Tez/FNIRS_bilgi.csv')
    df_age_sex.columns = ['Part_No', 'Gender', 'Age', 'Education']
    
    # Merging gender data with main data
    df_whole = pd.merge(df_age_sex, 
                          df,  
                          on ='Part_No', 
                          how ='inner')
    
    # Gender One Hot Encoding
    One_enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(One_enc.fit_transform(df_whole[['Gender']]).toarray())
    df_whole = df_whole.join(enc_df)
    df_whole.rename(columns = {0:'Male', 1:'Female'}, inplace = True)
    Z = df_whole.groupby(['Part_No'], as_index = False).mean()
    
    # GMM Clustering and its One Hot Encoding
    GMM_ = GMM(n_components = clusters, covariance_type='full' ).fit(Z.iloc[:,9:]) 
    label = GMM_.predict(Z.iloc[:,9:])
    Z['Clusters'] = label    
    Z = Z[['Part_No']].join(Z['Clusters'])
    df_whole = pd.merge(df_whole, 
                          Z, 
                          on ='Part_No', 
                          how ='inner')    
    One_enc_2 = OneHotEncoder(handle_unknown='ignore')
    enc_df_2 = pd.DataFrame(One_enc_2.fit_transform(df_whole[['Clusters']]).toarray())
    df_whole = df_whole.join(enc_df_2)
#     df_whole.rename(columns = {0:'Cluster 1', 1:'Cluster 2'}, inplace = True)
    
    # Dropping unnecassary columns as to make it same shape as other data
    df_whole.drop(['Clusters', 'Gender', 'Age', 'Education'], inplace = True, axis = 1)

    return df_whole

# Permutation Scores

In [ ]:
def permutation_test(fitted_model, X_test, Y_test, n_permutations):
  fitted_model.predict(X_test)
  score = fitted_model.score(X_test,Y_test)
  permutation_scores = []
  higher_count = 0
  for i in range(n_permutations):
    Y_test_perm = shuffle(Y_test,random_state=i) 
    perm_score = fitted_model.score(X_test,Y_test_perm)
    permutation_scores.append(perm_score)
    if perm_score >= score:
      higher_count += 1
  p_value = (1 + higher_count) / (1+ n_permutations)

  return score, permutation_scores, p_value


In [ ]:
def permutationTest(model, X_train, Y_train, cv, groups, n=50):
    score, perm_scores, pvalue = permutation_test_score(
    model, X_train, Y_train, groups=groups, scoring="accuracy", cv=cv, n_permutations=n)
    fig, ax = plt.subplots()

    ax.hist(perm_scores, bins=20, density=True)
    ax.axvline(score, ls="--", color="r")
    score_label = f"Score on original\ndata: {score:.2f}\n(p-value: {pvalue:.4f})"
    ax.text(0.7, 10, score_label, fontsize=12)
    ax.set_xlabel("F1 score")
    ax.set_ylabel("Probability")
    plt.show()
    return 
    

In [ ]:
def permutation_importance(model_, X_pca_train, Y_train):   
    # perform permutation importance
    results = permutation_importance(model_, X_pca_train, Y_train, scoring='accuracy')
    # get importance
    importance = results.importances_mean
    # summarize feature importance
    for i,v in enumerate(importance):
        print('Feature: %0d, Score: %.5f' % (i,v))
    # plot feature importance
    plt.bar([x for x in range(len(importance))], importance)
    plt.show()

# EXPLANIED VARIANCE RATIO AND FEATURE NUMBER

In [ ]:
def n_to_reach(df,featureCount, variance):
    
    print('Finding feature number to be reduced to given the explained variance ratio...')
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)
    X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:,1:-i], df.iloc[:,0], test_size=0.2, random_state=0)
        
    # finding n_components for maintaining specified variance
    # Standardization
    scaler = StandardScaler()

    X_train_scaled = scaler.fit_transform(X_train)

    X_test_scaled = scaler.transform(X_test)
    
    print("Starting PCA...")
    start = time.time()
    pca = PCA()    
    X_pca_train = pca.fit_transform(X_train_scaled)
    
    total_explained_variance = pca.explained_variance_ratio_.cumsum()
    n_over = len(total_explained_variance[total_explained_variance >= variance])
    n_to_reach = X_train.shape[1] - n_over + 1
    print("Number features: {}\tTotal Variance Explained: {}"
          .format(n_to_reach, total_explained_variance[n_to_reach-1]))
    return n_to_reach

In [ ]:
global lgb_param
global rfc_param
global svm_param
global knn_param
global xgb_param
global pos_weight
pos_weight = 0.88

lgb_param = {
                    'boosting_type': ['gbtree','dart'],
                    'num_leaves': [16,48,96],
                    'learning_rate': [0.01, 0.03, 0.05, 0.1],
                    'max_depth': [3,4,5],
                    'max_bin': [10,50,100],
#                     'n_estimators' : [500,1000]
#                 'boosting_type': ['gbdt', 'goss', 'dart'],
#                 'num_leaves': list(range(10, 20)),
#                 'learning_rate': [0.01, 0.03, 0.05, 0.1],
#                 'max_depth':list(range(1,12)),
#                 'max_bin': list(range(2, 30)),
                'num_iterations': [100,300,500],
                'early_stopping_rounds': [5,10],
                'scale_pos_weight': [pos_weight],
                'n_jobs' : [-1],
                'path_smooth': [3,4,5],
                'random_state': [0]
                     }
rfc_param =  {'bootstrap': [True],
                       'max_depth': [3, 4, 5],
                       'n_estimators': [300,500,1000],
                       'criterion' :['gini', 'entropy']
                     }
svm_param = {'C': [0.01, 0.1, 1, 3, 5],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']
                     }

knn_param = {'n_neighbors': list(range(3,21)),
              'metric': ['euclidean']
                   }

xgb_param = {'learning_rate':[0.01, 0.03, 0.05, 0.1],
                'max_depth':[3,4,5],
                'booster': ['gbtree','dart'],
                'random_state': [0],
                'n_jobs': [-1],
                'scale_pos_weight': [pos_weight],
                'gamma': [1, 5, 10],
                'n_estimators': [100,300,500],
                'min_child_weight': list(range(3,8)),
                'use_label_encoder': [False],
                'early_stopping_rounds':  [5,10],
                'objective': ['binary:logistic'], 
                'eval_metric': ['auc'],
                'verbosity': [0]
                     }

# ALL MODELS

In [ ]:
def main_model_wrapperMethod(df, model, model_name, featureCount, SMOTEBool=False, distance='euclidean', cv_splits=5, random_state=0, method='original', n_to_reach=0.99, groupBool=False,verbose=False, wrapper=True, cv_resultsBool=False):
    
    # df = pd.concat([df[df.columns[8:]], df[df.columns[0]]], ignore_index=True, axis=1)
    cols = df.columns.tolist()
    df = df[cols[8:]+[cols[0]]]
    i = df.shape[1] - (featureCount + 1)

    # Parameter tuning
    param_grid = {}
    if model_name == 'lgb':
      param_grid=lgb_param
    elif model_name == 'rfc':
      param_grid=rfc_param 
    elif model_name == 'SVM':
       param_grid=svm_param
    elif model_name == 'KNN':
      param_grid=knn_param
    elif model_name == 'XGB':
      param_grid=xgb_param

    # Standardization
    scaler = StandardScaler()
    
    # Normalization
#     scaler = MinMaxScaler()
    if groupBool:
      groups = df.Part_No.tolist()
    else:
      groups=None
    # df = df.iloc[:,:-1]
    X = df.iloc[:,1:]
    y = df.iloc[:,0]

    # Pipeline
    if method == 'original':
      # TODO: std by subject
      if wrapper:
        start = time.time()
        sfs = SFS(model,
            n_features_to_select=int(df.shape[1]/6),
            direction='forward',
            cv = cv_splits,
            n_jobs=-1)
        X = scaler.fit_transform(X)
        X = sfs.fit_transform(X, y)
        if verbose:
          print(f"Wrapper= {str(datetime.timedelta(seconds=(time.time()-start)))}")
      else:
        name = 'clf__'
        param_grid = {f"{name}{key}" : value for key, value in  param_grid.items()}
        pipe = Pipeline(steps=[('scaler', scaler),
                      ('clf', model)])
    elif method == 'PCA':
      start=time.time()
      pca = PCA(n_components=n_to_reach)
      # X = standardization_byParticipant(X)
      X = standardization_byRows(X)
      start = time.time()
      X = pca.fit_transform(X)
      if verbose:
        print(f"PCA= {str(datetime.timedelta(seconds=(time.time()-start)))}")
      if wrapper:
        start = time.time()
        sfs = SFS(model,
            n_features_to_select=int(2*len(pca.components_)/3),
            direction='forward',
            cv = cv_splits,
            n_jobs=-1)
        X = sfs.fit_transform(X, y)
        if verbose:
          print(f"Wrapper= {str(datetime.timedelta(seconds=(time.time()-start)))}")
    elif method == 'ISOMAP':
      X = scaler.fit_transform(X)
      n_to_reach = explained_variance(X, n_to_reach)
      iso = Isomap(n_components=n_to_reach)
      X = iso.fit_transform(X)
      if wrapper:
        start = time.time()
        sfs = SFS(model,
            n_features_to_select=int(2*len(n_to_reach)/3),
            direction='forward',
            cv = cv_splits,
            n_jobs=-1)
        X = sfs.fit_transform(X, y)
        if verbose:
          print(f"Wrapper= {str(datetime.timedelta(seconds=(time.time()-start)))}")
    elif method == 'TSNE':
      X = scaler.fit_transform(X)
      n_to_reach = explained_variance(X,n_to_reach)
      tsne = TSNE(n_components=n_to_reach, init='pca', random_state=0, method='exact', n_jobs=-1, learning_rate = 0.1, n_iter=500)
      X = tsne.fit_transform(X)
      if wrapper:
        start = time.time()
        sfs = SFS(model,
            n_features_to_select=int(2*len(n_to_reach)/3),
            direction='forward',
            cv = cv_splits,
            n_jobs=-1)
        X = sfs.fit_transform(X, y)
        if verbose:
          print(f"Wrapper= {str(datetime.timedelta(seconds=(time.time()-start)))}")
    
    #SMOTE for Train Data to Balance Dataset and also to Increase Data Size
    if SMOTEBool == True:
        print('SMOTE is being applied to the train set...')
        oversample = SMOTE()
        X, y = oversample.fit_resample(X, y)
    
    # K-fold cross validation
    start = time.time()
    if groupBool:
      cv = StratifiedGroupKFold(n_splits=cv_splits, random_state=1, shuffle=True)
      
    else:
      cv = StratifiedKFold(n_splits=cv_splits, random_state=1, shuffle=True)
    if cv_resultsBool:
      if method == 'TSNE' or method =='ISOMAP' or \
          method == 'PCA' or (method=='original' and wrapper==True):
        cv_results = cross_val_score(model, X, y, groups=groups, cv=cv, scoring='accuracy', n_jobs=-1)

      else:
        cv_results = cross_val_score(pipe, X, y, groups=groups, cv=cv, scoring='accuracy', n_jobs=-1)
      # print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
    else:
      cv_results = None

    if verbose:
      print(f"StratifiedGroupKFold = {str(datetime.timedelta(seconds=(time.time()-start)))}")

    # Trial for Mahalanobis distance metric for KNN -- seems worse than euclidean so commented
    # if model_name == 'KNN' and distance == 'mahalanobis':
        
    #     param_grid = {'n_neighbors': list(range(3,21)),
    #           'metric': ['mahalanobis'],
    #           'algorithm': ['brute'],
    #           'metric_params': [{'VI': np.linalg.inv(np.cov(X))}]
    #     }
    # GridSearch for Hyperparameters
    start = time.time()
    if method=='original' and wrapper==False:
      model_ = GridSearchCV(pipe,           
                            param_grid = param_grid,    
                            scoring='accuracy',         
                            cv=cv,                      
                            n_jobs=-1)
      
    else:
      model_ = GridSearchCV(model,           
                            param_grid = param_grid,    
                            scoring='accuracy',         
                            cv=cv,                      
                            n_jobs=-1)  
               
    model_.fit(X, y, groups=groups)

    if verbose:
      print(f"GridSearch = {str(datetime.timedelta(seconds=(time.time()-start)))}")
    
    if method =='PCA':
      try:
        sfs
        return [cv_results, sfs, pca, model_]
      except:
        return [cv_results, None, pca, model_]
    else:
      try:
        sfs
        return [cv_results, sfs, None, model_]
      except:
        return [cv_results, None, None, model_]

# MAIN MODEL

In [ ]:
def main_model_new(df, model, model_name, featureCount, SMOTEBool=False, distance='euclidean', cv_splits=5, random_state=0, method='original', n_to_reach=0.95):
    
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)

    # Group Split of Data into Train/Test
    # X = df.iloc[:,1:-i]
    # X.reset_index(inplace=True,drop=True)
    # Y = df.iloc[:,0]
    # Y.reset_index(inplace=True,drop=True)
    # groups = df.iloc[:,-1:] 
    # groups.reset_index(inplace=True,drop=True)

    # gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=0)
    # gss.get_n_splits()
    # for train_index, test_index in gss.split(X,Y, groups):
    #   X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      # Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    # X_train, X_test, Y_train, Y_test = train_test_split(
    # df.iloc[:,1:-i], df.iloc[:,0], test_size=0.20, random_state=random_state)
    print('MAIN RESULTS')
    # pos_weight = sum(Y_train[Y_train == 0]) / sum(Y_train[Y_train == 1])
    # train_weight = sum(1 if x == 0 else 0 for x in Y_train)\
    #                     /sum(1 if x == 1 else 0 for x in Y_train)

    # Parameter tuning
    param_grid = {}
    if model_name == 'lgb':
      param_grid=lgb_param
    elif model_name == 'rfc':
      param_grid=rfc_param 
    elif model_name == 'SVM':
       param_grid=svm_param
    elif model_name == 'KNN':
      param_grid=knn_param
    elif model_name == 'XGB':
      param_grid=xgb_param

    # Standardization
    scaler = StandardScaler()
    
    # Normalization
#     scaler = MinMaxScaler()
    
    
    # Pipeline
    if method == 'original':
      pipe = Pipeline([('scaler', scaler),
                      ('clf', model)])
    elif method == 'PCA':
      pipe = Pipeline([('scaler', scaler),
                       ('pca'), PCA(n_components=n_to_reach)
                      ('clf', model)])
    elif method == 'ISOMAP':
      pipe = Pipeline([('scaler', scaler),
                       ('iso'), Isomap(n_components=n_to_reach)
                      ('clf', model)])
    elif method == 'T-SNE':
      pipe = Pipeline([('scaler', scaler),
                       ('tsne'), TSNE(n_components=n_to_reach)
                      ('clf', model)])

   
    groups = df.iloc[:,-1]
    df = df.iloc[:,:-1]
    X = df.iloc[:,0]
    y = df.iloc[:,1:]

    #SMOTE for Train Data to Balance Dataset and also to Increase Data Size
    if SMOTEBool == True:
        print('SMOTE is being applied to the train set...')
        oversample = SMOTE()
        X, y = oversample.fit_resample(X, y)
    
    # K-fold cross validation
    print('Starting StratifiedGroupKFold...')
    start = time.time()
    cv = StratifiedGroupKFold(n_splits=cv_splits, random_state=1, shuffle=True)
    cv_results = cross_val_score(pipe, X, y, groups=groups, cv=cv, scoring='accuracy', n_jobs=-1)
    print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
    end = time.time()
    print(f"Total Time of StratifiedGroupKFold = {str(datetime.timedelta(seconds=(end-start)))}")

    # Trial for Mahalanobis distance metric for KNN -- seems worse than euclidean so commented
    if model_name == 'KNN' and distance == 'mahalanobis':
        
        param_grid = {'n_neighbors': list(range(3,21)),
              'metric': ['mahalanobis'],
              'algorithm': ['brute'],
              'metric_params': [{'VI': np.linalg.inv(np.cov(X))}]
        }
    # GridSearch for Hyperparameters
    print("Starting fitting into the model...")
    # start = time.time()
    # model_ = GridSearchCV(model,           
    #           param_grid = param_grid,    
    #           scoring='accuracy',         
    #           cv=cv,                      
    #           n_jobs=-1)             
    # model_.fit(X_train, Y_train, groups=groups)
    # end = time.time()
    # print(f"Total Time of model fit = {str(datetime.timedelta(seconds=(end-start)))}")


    return [X, y, groups, cv_results]

In [ ]:
def main_model(df, model, model_name, featureCount, SMOTEBool=False, distance='euclidean', cv_splits=5, random_state=0):
    
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)

    # Group Split of Data into Train/Test
    # X = df.iloc[:,1:-i]
    # X.reset_index(inplace=True,drop=True)
    # Y = df.iloc[:,0]
    # Y.reset_index(inplace=True,drop=True)
    # groups = df.iloc[:,-1:] 
    # groups.reset_index(inplace=True,drop=True)

    # gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=0)
    # gss.get_n_splits()
    # for train_index, test_index in gss.split(X,Y, groups):
    #   X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      # Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:,1:-i], df.iloc[:,0], test_size=0.20, random_state=random_state)
    print('MAIN RESULTS')
    # pos_weight = sum(Y_train[Y_train == 0]) / sum(Y_train[Y_train == 1])
    # train_weight = sum(1 if x == 0 else 0 for x in Y_train)\
    #                     /sum(1 if x == 1 else 0 for x in Y_train)

    # Parameter tuning
    param_grid = {}
    if model_name == 'lgb':
      param_grid=lgb_param
    elif model_name == 'rfc':
      param_grid=rfc_param 
    elif model_name == 'SVM':
       param_grid=svm_param
    elif model_name == 'KNN':
      param_grid=knn_param
    elif model_name == 'XGB':
      param_grid=xgb_param

    # Standardization
    scaler = StandardScaler()
    
    # Normalization
#     scaler = MinMaxScaler()
    
    
    # Pipeline
#     pipe = Pipeline([('scaler', scaler),
#                      ('clf', model)])

    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled)
    X_train_scaled.index = X_train.index
    X_train = pd.merge(X_train_scaled, df[df.columns[-i:]], left_index=True, right_index=True)
    X_train.columns = np.arange(X_train.shape[1])
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled)
    X_test_scaled.index = X_test.index
    X_test = pd.merge(X_test_scaled, df[df.columns[-i:]], left_index=True, right_index=True)
    X_test.columns = np.arange(X_test.shape[1])
    X_test = X_test.iloc[:,:-1]

   
    groups = X_train.iloc[:,-1]
    X_train = X_train.iloc[:,:-1]
    # groups = []
    # for index, row in X_train.iterrows():
    #   if row.iloc[-1] == 1:
    #     groups.append(1)
    #   elif row.iloc[-2] == 1:
    #     groups.append(2)
    #   elif row.iloc[-3] == 1:
    #     groups.append(3)
    # groups = pd.Series(groups)

    #SMOTE for Train Data to Balance Dataset and also to Increase Data Size
    if SMOTEBool == True:
        print('SMOTE is being applied to the train set...')
        oversample = SMOTE()
        X_train, Y_train = oversample.fit_resample(X_train, Y_train)
    
    # K-fold cross validation
    # print('Starting StratifiedGroupKFold...')
    # start = time.time()
    cv = StratifiedGroupKFold(n_splits=cv_splits, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, groups=groups, cv=cv, scoring='accuracy', n_jobs=-1)
    # print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
    # end = time.time()
    # print(f"Total Time of StratifiedGroupKFold = {str(datetime.timedelta(seconds=(end-start)))}")

    # Trial for Mahalanobis distance metric for KNN -- seems worse than euclidean so commented
    if model_name == 'KNN' and distance == 'mahalanobis':
        
        param_grid = {'n_neighbors': list(range(3,21)),
              'metric': ['mahalanobis'],
              'algorithm': ['brute'],
              'metric_params': [{'VI': np.linalg.inv(np.cov(X_train))}]
        }
    # GridSearch for Hyperparameters
    print("Starting fitting into the model...")
    start = time.time()
    model_ = GridSearchCV(model,           
              param_grid = param_grid,    
              scoring='accuracy',         
              cv=cv,                      
              n_jobs=-1)             
    model_.fit(X_train, Y_train, groups=groups)
    end = time.time()
    print(f"Total Time of model fit = {str(datetime.timedelta(seconds=(end-start)))}")

    # model.set_params(**model_.best_params_)
    # print(model.get_params())
    
    #permutationTest(model, X_train, Y_train, cv, groups)
    
#     # perform permutation importance
#     results = permutation_importance(model_, X_train, Y_train, scoring='accuracy')
#     # get importance
#     importance = results.importances_mean
#     # summarize feature importance
#     for i,v in enumerate(importance):
#         print('Feature: %0d, Score: %.5f' % (i,v))
#     # plot feature importance
#     plt.bar([x for x in range(len(importance))], importance)
#     plt.show()
    
    print("Tuned Hyperparameters :", model_.best_params_)
#     print("Tuned Best Accuracy :", model_.best_score_)
    
    # Predictions
    prediction = model_.predict(X_test)
    pred_prob = model_.predict_proba(X_test)
#     print(pred_prob)

    # keep probabilities for the positive outcome only
    pred_prob = pred_prob[:, 1]
#     print(pred_prob_pos)
    
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(Y_test, pred_prob)
    
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
#     print('Best Threshold = %f, G-Mean = %.3f' % (thresholds[ix], gmeans[ix]))
    print('Train Accuracy Score:', model_.score(X_train, Y_train))   
    
    # Probability of predictions adjusted to new threshold
    # predict_model = [1 if y >= thresholds[ix] else 0 for y in pred_prob]
    
    # Original threshold
    predict_model = [1 if y >= 0.5 else 0 for y in pred_prob]
    predict_model = np.array(predict_model).reshape(-1,1)

    print('Test Accuracy Score:', model_.score(X_test,Y_test))
    print('AUC Score:', roc_auc_score(Y_test, predict_model))
    print('F1 Score:', f1_score(Y_test, predict_model))
    print('Precision Score:', precision_score(Y_test, predict_model))
    print('Recall Score:', recall_score(Y_test, predict_model))
    print('\n')

#     print(classification_report(Y_test, predict_model))
#     print('Confusion Matrix:', '\n', pd.DataFrame(confusion_matrix(Y_test, predict_model)))
    
#     score_train = model.score(X_train, Y_train)
#     score_test = model.score(X_test, Y_test)
    
#     print(score_train, score_test)
    return [model_, X_test, Y_test, groups]

# MODEL PCA

In [ ]:
def model_pca(df, model, model_name, featureCount, n_to_reach, SMOTEBool=False):
    
    print('PCA RESULTS')
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)
    X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:,1:-i], df.iloc[:,0], test_size=0.2, random_state=0)



    param_grid = {}
    pos_weight = sum(Y_train[Y_train == 0]) / sum(Y_train[Y_train == 1])
    train_weight = sum(1 if x == 0 else 0 for x in Y_train)\
                        /sum(1 if x == 1 else 0 for x in Y_train)
    # Parameter tuning
    if model_name == 'lgb':
        
        param_grid = {
#                     'max_depth': [2, 3, 4],
#                     'num_leaves': [16,48,96],
#                     'max_bin': [500,1000],
#                     'boosting_type': ['dart'],
#                     'learning_rate': [0.01,0.03],
#                     'n_estimators' : [500,1000]
                'boosting_type': ['gbdt', 'goss', 'dart'],
                'num_leaves': list(range(10, 20)),
                'learning_rate': [0.01, 0.03, 0.05, 0.1],
                'num_iterations': list(range(50, 100)),
                'early_stopping_rounds': list(range(1, 10)),
                'scale_pos_weight': [pos_weight],
                'max_depth':list(range(1,12)),
                'max_bin': list(range(2, 30)),
                'n_jobs' : [-1],
                'path_smooth': list(range(2,15)),
                'random_state': [0]
                     }

    
    elif model_name == 'rfc':
        
        param_grid = {'bootstrap': [True],
                       'max_depth': [2, 3, 4],
                       'n_estimators': [100, 300, 500],
                       'criterion' :['gini', 'entropy']
                     }
        
    elif model_name == 'SVM':
        
        param_grid = {'C': [0.01, 0.1, 1, 3, 5],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']
                     }
        
    
    elif model_name == 'KNN':
        
        param_grid = {'n_neighbors': list(range(3,21)),
              'metric': ['euclidean']
                     }
        
    elif model_name == 'XGB':
        
        
        
        param_grid = {'learning_rate':[0.01, 0.03, 0.05, 0.1],
                'max_depth':[2,3,4],
                'booster': ('gbtree', 'gblinear'),
                'random_state': [0],
                'n_jobs': [-1],
                'scale_pos_weight': [pos_weight],
                'gamma': [1, 5, 10],
                'min_child_weight': list(range(3,8)),
                'use_label_encoder': [False],
                'early_stopping_rounds': list(range(3,10)),
                'objective': ['binary:logistic'], 
                'eval_metric': ['auc'],
                'verbosity': [0]
                     }
        
       
    # finding n_components for maintaining specified variance
        # Standardization
    scaler = StandardScaler()
    
    # Normalization
#     scaler = MinMaxScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
#     X_train_scaled = pd.DataFrame(X_train_scaled)
#     X_train_scaled.index = X_train.index
    
    
    X_test_scaled = scaler.transform(X_test)
#     X_test_scaled = pd.DataFrame(X_test_scaled)
#     X_test_scaled.index = X_test.index    
    
    print("Starting PCA...")
    start = time.time()
    # pca = PCA()    
    # X_pca_train = pca.fit_transform(X_train_scaled)
    
    # total_explained_variance = pca.explained_variance_ratio_.cumsum()
    # n_over = len(total_explained_variance[total_explained_variance >= variance])
    # n_to_reach = X_train.shape[1] - n_over + 1
    # print("Number features: {}\tTotal Variance Explained: {}"
    #       .format(n_to_reach, total_explained_variance[n_to_reach-1]))
    
    pca = PCA(n_components=n_to_reach)    
    X_pca_train = pca.fit_transform(X_train_scaled)
    X_pca_train = pd.DataFrame(X_pca_train)
    X_pca_train.index = X_train.index
    X_pca_train = pd.merge(X_pca_train, df[df.columns[-i:]], left_index=True, right_index=True)
    X_pca_train.columns = np.arange(X_pca_train.shape[1]) 

    X_pca_test = pca.transform(X_test_scaled)
    X_pca_test = pd.DataFrame(X_pca_test)
    X_pca_test.index = X_test.index
    X_pca_test = pd.merge(X_pca_test, df[df.columns[-i:]], left_index=True, right_index=True)
    X_pca_test.columns = np.arange(X_pca_test.shape[1])
    X_pca_test = X_pca_test.iloc[:,:-1]
    
    groups = X_pca_train.iloc[:,-1]
    X_pca_train = X_pca_train.iloc[:,:-1]
    end = time.time()
    print(f"Total Time of PCA = {str(datetime.timedelta(seconds=(end-start)))}")

    #SMOTE for Train Data to Balance Dataset and also to Increase Data Size
    if SMOTEBool == True:
        oversample = SMOTE()
        X_pca_train, Y_train = oversample.fit_resample(X_pca_train, Y_train)
    
    # K-fold cross validation
    cv = StratifiedGroupKFold(n_splits=5, random_state=1, shuffle=True)
#     cv_results = cross_val_score(model, X_pca_train, Y_train, groups=groups, cv=cv, scoring='accuracy')
#     print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
        
    # GridSearch for Hyperparameters
    model_ = GridSearchCV(model,           # model
              param_grid = param_grid,    # hyperparameters
              scoring='accuracy',         # metric for scoring
              cv=cv,                      # number of folds
              n_jobs=-1)
    print("Starting fitting into the model...")
    start = time.time()          
    model_.fit(X_pca_train,Y_train, groups=groups)
    end = time.time()
    print(f"Total Time of model fit = {str(datetime.timedelta(seconds=(end-start)))}")

    
    model.set_params(**model_.best_params_)
    print(model.get_params())
    #permutationTest(model, X_pca_train, Y_train, cv, groups)
    
#     # perform permutation importance
#     results = permutation_importance(model_, X_pca_train, Y_train, scoring='accuracy')
#     # get importance
#     importance = results.importances_mean
#     # summarize feature importance
#     for i,v in enumerate(importance):
#         print('Feature: %0d, Score: %.5f' % (i,v))
#     # plot feature importance
#     plt.bar([x for x in range(len(importance))], importance)
#     plt.show()
    
    
    print("Tuned Hyperparameters :", model_.best_params_)
#     print("Tuned Best Accuracy :", model_.best_score_)
    
    # Predictions
    pred_prob = model_.predict_proba(X_pca_test)
#     print(pred_prob)

    # keep probabilities for the positive outcome only
    pred_prob = pred_prob[:, 1]
#     print(pred_prob_pos)
    
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(Y_test, pred_prob)
    
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
#     print('Best Threshold = %f, G-Mean = %.3f' % (thresholds[ix], gmeans[ix]))
    print('Train Accuracy Score:', model_.score(X_pca_train, Y_train))   

    # Probability of predictions adjusted to new threshold
    predict_model = [1 if y >= thresholds[ix] else 0 for y in pred_prob]
    
    # Original threshold
#     predict_model = [1 if y >= 0.5 else 0 for y in pred_prob]
#     print(predict_model)

    
    #print(predict_lgb)
    print('Test Accuracy Score:', accuracy_score(Y_test, predict_model))
    print('AUC Score:', roc_auc_score(Y_test, predict_model))
    print('F1 Score:', f1_score(Y_test, predict_model))
    print('Precision Score:', precision_score(Y_test, predict_model))
    print('Recall Score:', recall_score(Y_test, predict_model))
    print('\n')
#     print(classification_report(Y_test, predict_model))
#     print('Confusion Matrix:', '\n', pd.DataFrame(confusion_matrix(Y_test, predict_model)))
    
#     score_train = model.score(X_train, Y_train)
#     score_test = model.score(X_test, Y_test)
    
#     print(score_train, score_test)
    return (f1_score(Y_test, predict_model), model_, X_pca_train, Y_train, cv, groups)

# MODEL TSNE

In [ ]:
def model_t_sne(df, model, model_name, featureCount, n_to_reach, SMOTEBool=False):
    
    # Standardization
    scaler = StandardScaler()
    print('TSNE RESULTS')
     # Test/Train Split
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)
    X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:,1:-i], df.iloc[:,0], test_size=0.2, random_state=0)
#     print(X_train)

    # Normalization
#     scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # pca = PCA()    
    # X_tsne_train = pca.fit_transform(X_train_scaled)

    
    # total_explained_variance = pca.explained_variance_ratio_.cumsum()
    # n_over = len(total_explained_variance[total_explained_variance >= variance])
    # n_to_reach = X_train.shape[1] - n_over + 1
    # print("Number features: {}\tTotal Variance Explained: {}"
    #       .format(n_to_reach, total_explained_variance[n_to_reach-1]))
    
    # T-SNE
    print("Starting TSNE...")
    start = time.time()    
    tsne = TSNE(n_components=n_to_reach, 
                init='pca', random_state=0, method='exact', n_jobs=-1, learning_rate = 0.1, n_iter=2000)
    X_tsne_train = tsne.fit_transform(X_train_scaled)
    X_tsne_train = pd.DataFrame(X_tsne_train)
    X_tsne_train.index = X_train.index
    X_tsne_train = pd.merge(X_tsne_train, df[df.columns[-i:]], left_index=True, right_index=True)
    X_tsne_train.columns = np.arange(X_tsne_train.shape[1]) 


    X_tsne_test = tsne.fit_transform(X_test_scaled)
    X_tsne_test = pd.DataFrame(X_tsne_test)
    X_tsne_test.index = X_test.index
    X_tsne_test = pd.merge(X_tsne_test, df[df.columns[-i:]], left_index=True, right_index=True)
    X_tsne_test.columns = np.arange(X_tsne_test.shape[1])
    X_tsne_test = X_tsne_test.iloc[:,:-1]
    
    groups = X_tsne_train.iloc[:,-1]
    X_tsne_train = X_tsne_train.iloc[:,:-1]
    end = time.time()
    print(f"Total Time of TSNE = {str(datetime.timedelta(seconds=(end-start)))}")
    
    #SMOTE for Train Data to Balance Dataset and also to Increase Data Size
    if SMOTEBool == True:
        oversample = SMOTE()
        X_tsne_train, Y_train = oversample.fit_resample(X_tsne_train, Y_train)
    
    
    param_grid = {}
    pos_weight = sum(Y_train[Y_train == 0]) / sum(Y_train[Y_train == 1])
    train_weight = sum(1 if x == 0 else 0 for x in Y_train)\
                        /sum(1 if x == 1 else 0 for x in Y_train)
    # Parameter tuning
    if model_name == 'lgb':
        
        param_grid = {
#                     'max_depth': [2, 3, 4],
#                     'num_leaves': [16,48,96],
#                     'max_bin': [500,1000],
#                     'boosting_type': ['dart'],
#                     'learning_rate': [0.01,0.03],
#                     'n_estimators' : [500,1000]
                'boosting_type': ['gbdt', 'goss', 'dart'],
                'num_leaves': list(range(10, 20)),
                'learning_rate': [0.01, 0.03, 0.05, 0.1],
                'num_iterations': list(range(50, 100)),
                'early_stopping_rounds': list(range(1, 10)),
                'scale_pos_weight': [pos_weight],
                'max_depth':list(range(1,12)),
                'max_bin': list(range(2, 30)),
                'n_jobs' : [-1],
                'path_smooth': list(range(2,15)),
                'random_state': [0]
                     }

    
    elif model_name == 'rfc':
        
        param_grid = {'bootstrap': [True],
                       'max_depth': [2, 3, 4],
                       'n_estimators': [100, 300, 500],
                       'criterion' :['gini', 'entropy']
                     }
        
    elif model_name == 'SVM':
        
        param_grid = {'C': [0.01, 0.1, 1, 3, 5],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']
                     }
        
    
    elif model_name == 'KNN':
        
        param_grid = {'n_neighbors': list(range(3,21)),
              'metric': ['euclidean']
                     }
        
    elif model_name == 'XGB':
        
        
        
        param_grid = {'learning_rate':[0.01, 0.03, 0.05, 0.1],
                'max_depth':[2,3,4],
                'booster': ('gbtree', 'gblinear'),
                'random_state': [0],
                'n_jobs': [-1],
                'scale_pos_weight': [pos_weight],
                'gamma': [1, 5, 10],
                'min_child_weight': list(range(3,8)),
                'use_label_encoder': [False],
                'early_stopping_rounds': list(range(3,10)),
                'objective': ['binary:logistic'], 
                'eval_metric': ['auc'],
                'verbosity': [0]
                     }

    # K-fold cross validation
    # print('Starting StratifiedGroupKFold...')
    cv = StratifiedGroupKFold(n_splits=5, random_state=1, shuffle=True)
#     cv_results = cross_val_score(model, X_tsne_train, Y_train,groups=groups, cv=cv, scoring='accuracy')
#     print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
    
#     permutationTest(model, X_tsne_train, Y_train, cv, groups)
    
    # GridSearch for Hyperparameters
    print("Starting fitting into the model...")
    start = time.time()
    model_ = GridSearchCV(model,           # model
              param_grid = param_grid,    # hyperparameters
              scoring='accuracy',         # metric for scoring
              cv=cv,                    # number of folds
              n_jobs=-1)             
    model_.fit(X_tsne_train,Y_train, groups=groups)
    end = time.time()
    print(f"Total Time of model fit = {str(datetime.timedelta(seconds=(end-start)))}")
    
#     # perform permutation importance
#     results = permutation_importance(model_, X_tsne_train, Y_train, scoring='accuracy')
#     # get importance
#     importance = results.importances_mean
#     # summarize feature importance
#     for i,v in enumerate(importance):
#         print('Feature: %0d, Score: %.5f' % (i,v))
#     # plot feature importance
#     plt.bar([x for x in range(len(importance))], importance)
#     plt.show()
    
    
    print("Tuned Hyperparameters :", model_.best_params_)
    
    model.set_params(**model_.best_params_)
    print(model.get_params())
    
    #permutationTest(model, X_tsne_train, Y_train, cv, groups)
    
#     print("Tuned Best Accuracy :", model_.best_score_)
    
    # Predictions
    pred_prob = model_.predict_proba(X_tsne_test)
#     print(pred_prob)

    # keep probabilities for the positive outcome only
    pred_prob = pred_prob[:, 1]
#     print(pred_prob_pos)
    
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(Y_test, pred_prob)
    
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
#     print('Best Threshold = %f, G-Mean = %.3f' % (thresholds[ix], gmeans[ix]))
    print('Train Accuracy Score:', model_.score(X_tsne_train, Y_train))

    # Probability of predictions adjusted to new threshold
    predict_model = [1 if y >= thresholds[ix] else 0 for y in pred_prob]
    
    # Original threshold
#     predict_model = [1 if y >= 0.5 else 0 for y in pred_prob]
#     print(predict_model)

    
    #print(predict_lgb)
    print('Test Accuracy Score:', accuracy_score(Y_test, predict_model))
    print('AUC Score:', roc_auc_score(Y_test, predict_model))
    print('F1 Score:', f1_score(Y_test, predict_model))
    print('Precision Score:', precision_score(Y_test, predict_model))
    print('Recall Score:', recall_score(Y_test, predict_model))
    print('\n')
#     print(classification_report(Y_test, predict_model))
#     print('Confusion Matrix:', '\n', pd.DataFrame(confusion_matrix(Y_test, predict_model)))
    
#     score_train = model.score(X_train, Y_train)
#     score_test = model.score(X_test, Y_test)
    
#     print(score_train, score_test)
    return (f1_score(Y_test, predict_model), model_, X_tsne_train,Y_train, cv, groups)

# MODEL ISOMAP

In [ ]:
def model_isomap(df, model, model_name, featureCount, n_to_reach, SMOTEBool=False):
    
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)
    X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:,1:-i], df.iloc[:,0], test_size=0.2, random_state=0)

    print('ISOMAP RESULTS')

    

    param_grid = {}
    pos_weight = sum(Y_train[Y_train == 0]) / sum(Y_train[Y_train == 1])
    train_weight = sum(1 if x == 0 else 0 for x in Y_train)\
                        /sum(1 if x == 1 else 0 for x in Y_train)
    # Parameter tuning
    if model_name == 'lgb':
        
        param_grid = {
#                     'max_depth': [2, 3, 4],
#                     'num_leaves': [16,48,96],
#                     'max_bin': [500,1000],
#                     'boosting_type': ['dart'],
#                     'learning_rate': [0.01,0.03],
#                     'n_estimators' : [500,1000]
                'boosting_type': ['gbdt', 'goss', 'dart'],
                'num_leaves': list(range(10, 20)),
                'learning_rate': [0.01, 0.03, 0.05, 0.1],
                'num_iterations': list(range(50, 100)),
                'early_stopping_rounds': list(range(1, 10)),
                'scale_pos_weight': [pos_weight],
                'max_depth':list(range(1,12)),
                'max_bin': list(range(2, 30)),
                'n_jobs' : [-1],
                'path_smooth': list(range(2,15)),
                'random_state': [0]
                     }

    
    elif model_name == 'rfc':
        
        param_grid = {'bootstrap': [True],
                       'max_depth': [2, 3, 4],
                       'n_estimators': [100, 300, 500],
                       'criterion' :['gini', 'entropy']
                     }
        
    elif model_name == 'SVM':
        
        param_grid = {'C': [0.01, 0.1, 1, 3, 5],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']
                     }
        
    
    elif model_name == 'KNN':
        
        param_grid = {'n_neighbors': list(range(3,21)),
              'metric': ['euclidean']
                     }
        
    elif model_name == 'XGB':
        
        
        
        param_grid = {'learning_rate':[0.01, 0.03, 0.05, 0.1],
                'max_depth':[2,3,4],
                'booster': ('gbtree', 'gblinear'),
                'random_state': [0],
                'n_jobs': [-1],
                'scale_pos_weight': [pos_weight],
                'gamma': [1, 5, 10],
                'min_child_weight': list(range(3,8)),
                'use_label_encoder': [False],
                'early_stopping_rounds': list(range(3,10)),
                'objective': ['binary:logistic'], 
                'eval_metric': ['auc'],
                'verbosity': [0]
                     }
        
    
    # Standardization
    scaler = StandardScaler()
    
    # Normalization
#     scaler = MinMaxScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # pca = PCA()    
    # X_iso_train = pca.fit_transform(X_train_scaled)
    
    # total_explained_variance = pca.explained_variance_ratio_.cumsum()
    # n_over = len(total_explained_variance[total_explained_variance >= variance])
    # n_to_reach = X_train.shape[1] - n_over + 1
    # print("Number features: {}\tTotal Variance Explained: {}"
    #       .format(n_to_reach, total_explained_variance[n_to_reach-1]))
    
    print("Starting ISOMAP...")
    start = time.time()
    iso = Isomap(n_components=n_to_reach)    
    X_iso_train = iso.fit_transform(X_train_scaled)
    X_iso_train = pd.DataFrame(X_iso_train)
    X_iso_train.index = X_train.index
    X_iso_train = pd.merge(X_iso_train, df[df.columns[-i:]], left_index=True, right_index=True)
    X_iso_train.columns = np.arange(X_iso_train.shape[1]) 


    X_iso_test = iso.transform(X_test_scaled)
    X_iso_test = pd.DataFrame(X_iso_test)
    X_iso_test.index = X_test.index
    X_iso_test = pd.merge(X_iso_test, df[df.columns[-i:]], left_index=True, right_index=True)
    X_iso_test.columns = np.arange(X_iso_test.shape[1])
    X_iso_test = X_iso_test.iloc[:,:-1]
    
    groups = X_iso_train.iloc[:,-1]
    X_iso_train = X_iso_train.iloc[:,:-1]
    end = time.time()
    print(f"Total Time of ISOMAP = {str(datetime.timedelta(seconds=(end-start)))}")

    #SMOTE for Train Data to Balance Dataset and also to Increase Data Size
    if SMOTEBool == True:
        oversample = SMOTE()
        X_iso_train, Y_train = oversample.fit_resample(X_iso_train, Y_train)
    
    # K-fold cross validation
    # print('Starting StratifiedGroupKFold...')
    cv = StratifiedGroupKFold(n_splits=5, random_state=1, shuffle=True)
#     cv_results = cross_val_score(model, X_iso_train, Y_train, groups=groups, cv=cv, scoring='accuracy')
#     print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
    
#     permutationTest(model, X_iso_train, Y_train, cv, groups)
    
    # GridSearch for Hyperparameters
    print("Starting fitting into the model...")
    start = time.time()
    model_ = GridSearchCV(model,           # model
              param_grid = param_grid,    # hyperparameters
              scoring='accuracy',         # metric for scoring
              cv=cv,                      # number of folds
              n_jobs=-1)             
    model_.fit(X_iso_train,Y_train, groups=groups)
    end = time.time()
    print(f"Total Time of model fit = {str(datetime.timedelta(seconds=(end-start)))}")


    model.set_params(**model_.best_params_)
    print(model.get_params())
    
    #permutationTest(model, X_iso_train, Y_train, cv, groups)
    
#     # perform permutation importance
#     results = permutation_importance(model_, X_iso_train, Y_train, scoring='accuracy')
#     # get importance
#     importance = results.importances_mean
#     # summarize feature importance
#     for i,v in enumerate(importance):
#         print('Feature: %0d, Score: %.5f' % (i,v))
#     # plot feature importance
#     plt.bar([x for x in range(len(importance))], importance)
#     plt.show()
    
    
    print("Tuned Hyperparameters :", model_.best_params_)
#     print("Tuned Best Accuracy :", model_.best_score_)
    
    # Predictions
    pred_prob = model_.predict_proba(X_iso_test)
#     print(pred_prob)

    # keep probabilities for the positive outcome only
    pred_prob = pred_prob[:, 1]
#     print(pred_prob_pos)
    
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(Y_test, pred_prob)
    
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
#     print('Best Threshold = %f, G-Mean = %.3f' % (thresholds[ix], gmeans[ix]))
    print('Train Accuracy Score:', model_.score(X_iso_train, Y_train))   
  
    # Probability of predictions adjusted to new threshold
    predict_model = [1 if y >= thresholds[ix] else 0 for y in pred_prob]
    
    # Original threshold
#     predict_model = [1 if y >= 0.5 else 0 for y in pred_prob]
#     print(predict_model)

    
    #print(predict_lgb)
    print('Test Accuracy Score:', accuracy_score(Y_test, predict_model))
    print('AUC Score:', roc_auc_score(Y_test, predict_model))
    print('F1 Score:', f1_score(Y_test, predict_model))
    print('Precision Score:', precision_score(Y_test, predict_model))
    print('Recall Score:', recall_score(Y_test, predict_model))
    print('\n')
#     print(classification_report(Y_test, predict_model))
#     print('Confusion Matrix:', '\n', pd.DataFrame(confusion_matrix(Y_test, predict_model)))
    
#     score_train = model.score(X_train, Y_train)
#     score_test = model.score(X_test, Y_test)
    
#     print(score_train, score_test)
    return (f1_score(Y_test, predict_model), model_, X_iso_train, Y_train, cv, groups)

In [ ]:
def main_model_WithinEachIndividual(df, model, model_name, featureCount, SMOTEBool=False):
    
 
    df = pd.concat([df[df.columns[8:]], df[df.columns[6]]], ignore_index=True, axis=1)
    i = df.shape[1] - (featureCount + 1)
    print(i)
    X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:,1:-i], df.iloc[:,0], test_size=0.20, random_state=0)
    print('MAIN RESULTS')
    
    param_grid = {}
    
    # Parameter tuning
    if model_name == 'lgb':
        
        train_weight = sum(1 if x == 0 else 0 for x in Y_train)\
                        /sum(1 if x == 1 else 0 for x in Y_train)

        param_grid = {'max_depth': [2, 3, 4],
                    'num_leaves': [16,48,96],
                    'max_bin': [500,1000],
                    'boosting_type': ['dart'],
                    'learning_rate': [0.01,0.03],
                    'n_estimators' : [500,1000]
                     }
    
    elif model_name == 'rfc':
        
        param_grid = {'bootstrap': [True],
                       'max_depth': [2, 3, 4],
                       'n_estimators': [100, 300, 500],
                       'criterion' :['gini', 'entropy']
                     }
        
    elif model_name == 'SVM':
        
        param_grid = {'C': [0.01, 0.1, 1, 3, 5],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']
                     }
        
    
    elif model_name == 'KNN':
        
        param_grid = {'n_neighbors': list(range(3,21)),
              'metric': ['euclidean']
                     }
        
    elif model_name == 'XGB':
        
        param_grid = {'learning_rate':[0.01, 0.03, 0.05, 0.1],
                'max_depth':[2,3,4],
                'booster': ('gbtree', 'gblinear'),
                'random_state': [0],
                'n_jobs': [-1],
                'gamma': [1, 5, 10],
                'min_child_weight': list(range(3,8)),
                'use_label_encoder': [False],
                'objective': ['binary:logistic'], 
                'eval_metric': ['error'],
                'verbosity': [0]
                     } 
        
        
        
       
    # Standardization
    scaler = StandardScaler()
    
    # Normalization
#     scaler = MinMaxScaler()
    
    
    # Pipeline
#     pipe = Pipeline([('scaler', scaler),
#                      ('clf', model)])

    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled)
    X_train_scaled.index = X_train.index
    X_train = pd.merge(X_train_scaled, df[df.columns[-i:]], left_index=True, right_index=True)
    X_train.columns = np.arange(X_train.shape[1])
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled)
    X_test_scaled.index = X_test.index
    X_test = pd.merge(X_test_scaled, df[df.columns[-i:]], left_index=True, right_index=True)
    X_test.columns = np.arange(X_test.shape[1])
    X_test = X_test.iloc[:,:-1]
    
    groups = X_train.iloc[:,-1]
    X_train = X_train.iloc[:,:-1]
    
    if SMOTEBool == True:
        oversample = SMOTE()
        X_train, Y_train = oversample.fit_resample(X_train, Y_train)
    
    # K-fold cross validation
    cv = StratifiedGroupKFold(n_splits=5, random_state=1, shuffle=True)
#     cv_results = cross_val_score(model, X_train, Y_train, groups=groups, cv=cv, scoring='accuracy')
#     print('Cross Validation Accuracy: %.3f (%.3f)' % (np.mean(cv_results), np.std(cv_results)))
    
    
#     permutationTest(model, X_train, Y_train, cv, groups)
       
    
    # Randomized Search for Hyperparameters
#     model_ = RandomizedSearchCV(pipe, param_grid, scoring='accuracy',cv=cv, random_state=0)
#     model_.fit(X_train,Y_train)
    
    # GridSearch for Hyperparameters
    
    model_ = GridSearchCV(model,           # model
              param_grid = param_grid,    # hyperparameters
              scoring='accuracy',         # metric for scoring
              cv=cv                      # number of folds
              )             
    model_.fit(X_train, Y_train, groups=groups)
    
    model.set_params(**model_.best_params_)
    print(model.get_params())
    permutationTest(model, X_train, Y_train, cv, groups)
    
#     # perform permutation importance
#     results = permutation_importance(model_, X_train, Y_train, scoring='accuracy')
#     # get importance
#     importance = results.importances_mean
#     # summarize feature importance
#     for i,v in enumerate(importance):
#         print('Feature: %0d, Score: %.5f' % (i,v))
#     # plot feature importance
#     plt.bar([x for x in range(len(importance))], importance)
#     plt.show()
    
    print("Tuned Hyperparameters :", model_.best_params_)
#     print("Tuned Best Accuracy :", model_.best_score_)
    
    # Predictions
    pred_prob = model_.predict_proba(X_test)
#     print(pred_prob)

    # keep probabilities for the positive outcome only
    pred_prob = pred_prob[:, 1]
#     print(pred_prob_pos)
    
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(Y_test, pred_prob)
    
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
#     print('Best Threshold = %f, G-Mean = %.3f' % (thresholds[ix], gmeans[ix]))
    print('Train Accuracy Score:', model_.score(X_train, Y_train))   
    
    # Probability of predictions adjusted to new threshold
    predict_model = [1 if y >= thresholds[ix] else 0 for y in pred_prob]
    
    # Original threshold
#     predict_model = [1 if y >= 0.5 else 0 for y in pred_prob]
#     print(predict_model)

    
    #print(predict_lgb)
    print('Test Accuracy Score:', accuracy_score(Y_test, predict_model))
    print('AUC Score:', roc_auc_score(Y_test, predict_model))
    print('F1 Score:', f1_score(Y_test, predict_model))
    print('Precision Score:', precision_score(Y_test, predict_model))
    print('Recall Score:', recall_score(Y_test, predict_model))
    print('\n')
#     print(classification_report(Y_test, predict_model))
#     print('Confusion Matrix:', '\n', pd.DataFrame(confusion_matrix(Y_test, predict_model)))
    
#     score_train = model.score(X_train, Y_train)
#     score_test = model.score(X_test, Y_test)
    
#     print(score_train, score_test)
    return f1_score(Y_test, predict_model)